In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.metrics.pairwise import  laplacian_kernel
from sklearn.decomposition import PCA

from scipy.integrate import quad
from scipy.interpolate import interp1d
from joblib import Parallel, delayed

from WHIDataModule import *
from estimators import *
from utils import *
from mmr_utils import *

In [2]:
def run_single_mmr(df_pca, train_index, mmr_index, mmr_keys, jD):

    df_train = df_pca.iloc[train_index, :].copy()
    df_mmr = df_pca.iloc[mmr_index, :].copy()

    # fit a model for the selection score P(S=1|X) and clip the population based on it
    X, y = df_train[jD["cov_list"]], df_train["S"]

    logreg = LogisticRegressionCV(cv=5, random_state=42, penalty='l2', solver='lbfgs')
    logreg.fit(X, y)

    df_mmr['P(S=1|X)'] = logreg.predict_proba(df_mmr[jD["cov_list"]])[:,1]
    df_mmr = df_mmr.loc[(df_mmr['P(S=1|X)'] > 0.05) \
                        & (df_mmr['P(S=1|X)']< 0.95)].reset_index(drop=True)

    # fit a model for the propensity scores P(A=1|X,S) and clip the population based on them
    for s in range(2):
        X = df_train.query(f"S=={s}")[jD["cov_list"]]
        y = df_train.query(f"S=={s}")["A"]

        logreg = LogisticRegressionCV(cv=5, random_state=42, penalty='l2', solver='lbfgs')
        logreg.fit(X, y)

        df_mmr.loc[df_mmr['S']==s, 'P(A=1|X,S)'] = logreg.predict_proba(df_mmr.loc[df_mmr['S']==s, jD["cov_list"]])[:,1]

    df_mmr = df_mmr.loc[(df_mmr['P(A=1|X,S)'] > 0.05) \
                        & (df_mmr['P(A=1|X,S)']< 0.95)].reset_index(drop=True)

    # fit response surface signals with the imputed data
    mu_regressor = {}
    for s in range(2):
        for a in range(2):
            mu_regressor[f'S{s}_A{a}'] =\
            mu_est_baseline(df_train.query(f'S=={s} & A=={a}').copy(), 'T', jD["cov_list"], model_name='linear')

        df_mmr.loc[df_mmr["S"]==s, 'mu(Y|X,S,A=0)'] = mu_regressor[f'S{s}_A0'].predict(df_mmr.loc[df_mmr["S"]==s, jD["cov_list"]])
        df_mmr.loc[df_mmr["S"]==s, 'mu(Y|X,S,A=1)'] = mu_regressor[f'S{s}_A1'].predict(df_mmr.loc[df_mmr["S"]==s, jD["cov_list"]])

    # fit the survival curves
    Gb_C, Fb_Y = est_surv_whi(df_train, 'coxph', jD["cov_list"], downsample=10)
    df_mmr['Gb(T|X,S,A)'] = df_mmr.apply(lambda r:\
        eval_surv_(Gb_C[f"t_S{int(r['S'])}_A{int(r['A'])}"], Gb_C[f"St_S{int(r['S'])}_A{int(r['A'])}"], r['T']), axis=1)

    ipw_est(df_mmr, S=0, baseline='impute')  # censored observations are IMPUTED
    ipw_est(df_mmr, S=1, baseline='impute')  # censored observations are IMPUTED

    ipcw_est(df_mmr, S=0)
    ipcw_est(df_mmr, S=1)

    dr_est(df_mmr, S=0, baseline='impute')  # censored observations are IMPUTED
    dr_est(df_mmr, S=1, baseline='impute')  # censored observations are IMPUTED

    cdr_est(df_mmr, jD['cov_list'], Gb_C, Fb_Y, S=0, mis_spec='None')  
    cdr_est(df_mmr, jD['cov_list'], Gb_C, Fb_Y, S=1, mis_spec='None')  

    mmr_stats = np.zeros((len(mmr_keys), 2))

    # run mmr test and record the results
    for kind, key in enumerate(mmr_keys):
        signal0, signal1 = jD['test_signals'][key][0], jD['test_signals'][key][1]
        mmr_stats[kind, 0], mmr_stats[kind, 1] =\
                mmr_test(df_mmr, jD['cov_list'], jD['B'], laplacian_kernel, signal0, signal1)
        
    return mmr_stats

In [4]:
df_combined = pd.read_csv("/data/whi/data/main_study/processed/whi_combined_nocolinear.csv")  # combined dataset  # combined dataset
treat_event_arr = np.array(df_combined.query("S == 0 & A == 0 & Delta == 1").index)
sb_arr = np.random.choice(treat_event_arr, size=int(len(treat_event_arr) / 10))
df_combined.drop(sb_arr, inplace=True)
df_combined = df_combined.reset_index(drop=True)
orig_cov_list = list(df_combined.columns[4:])  # original set of covariates
print(f'Original data size: {len(df_combined)}') 

mmr_keys = ["IPW-Impute", "DR-Impute", "IPCW", "CDR"]  # The signals to run the falsification test with
jD = read_json_whi('whi/whi.json', mmr_keys)  # read the experiment settings

PC_dim_list = [350]  # num. Principal components to experiment with 
num_folds = 10  # we train the models using 9 folds and run the MMR test with the predictions in the remaining fold

mmr_global_stats = np.zeros((len(PC_dim_list), num_folds, len(mmr_keys), 2))  # store results and p-val for each mmr test

for pcind, PC_dim in enumerate(PC_dim_list):
    t1 = time()
    df_pca = df_combined.copy()
    df_pca = df_pca.sample(frac=1, random_state=1337)  # shuffle the rows so that S=0 and S=1 are not stacked

    pca = PCA(n_components=PC_dim)
    principalComponents = pca.fit_transform(df_pca[orig_cov_list])
    print(f"Var. explained by {PC_dim} PCs: {sum(pca.explained_variance_ratio_):.2f}")

    df_pca_new_cols = pd.DataFrame(data=sm.add_constant(principalComponents, prepend=True), columns=[f"PC{i}" for i in range(PC_dim + 1)])
    df_pca = df_pca.drop(orig_cov_list, axis=1)
    df_pca = pd.concat([df_pca, df_pca_new_cols], axis=1)

    jD["cov_list"] = [f"PC{i}" for i in range(PC_dim + 1)]
    kf = KFold(n_splits=num_folds)
    
    local_mmr_res = Parallel(n_jobs=20)(delayed(run_single_mmr)(df_pca.copy(), train_index, mmr_index, mmr_keys.copy(), jD.copy()) \
                                 for (train_index, mmr_index) in kf.split(df_pca))

    # run mmr test and record the results
    for iter in range(num_folds):
        for kind, key in enumerate(mmr_keys):
            mmr_global_stats[pcind, iter, kind, 0] = local_mmr_res[iter][kind][0]
            mmr_global_stats[pcind, iter, kind, 1] = local_mmr_res[iter][kind][1]

    time_elapsed = time() - t1
    print(f"PC: {PC_dim} completed in {time_elapsed:.1f} seconds")

pc_vals = [f"PC {pc}" for pc in PC_dim_list]

avg_reject = np.mean(mmr_global_stats[:,:,:,0], axis=1)
avg_pval = np.mean(mmr_global_stats[:,:,:,1], axis=1)

reject_df = pd.DataFrame(avg_reject, columns=mmr_keys, index=pc_vals)
pval_df = pd.DataFrame(avg_pval, columns=mmr_keys, index=pc_vals)

reject_df.to_csv(f"results/whi90/sba0_1_reject_rates_{num_folds}.csv")
pval_df.to_csv(f"results/whi90/sba0_1_pvals_avg_{num_folds}.csv")
np.save(f"results/whi90/sba0_1_all_results_{num_folds}.npy", mmr_global_stats) 

Original data size: 49968
Var. explained by 350 PCs: 0.90
PC: 350 completed in 264.3 seconds


In [5]:
df_combined

,S,A,Delta,T,const,AGE,MENOPSEA,ANYMENSA,MENO,TEPIWK,...,TKRETIN_Yes,TKSELEN_Yes,TKZINC_Yes,SYSTOL_<=120,SYSTOL_>140,DIASTOL_>=90,BMIC_Obesity I (30.0 - 34.9),BMIC_Obesity II (35.0 - 39.9),BMIC_Overweight (25.0 - 29.9),BMIC_Underweight (< 18.5)
0,0,1,0,2190.0,1.0,0.525013,0.948703,0.652583,0.828300,1.411645,...,0,0,0,1,0,0,0,0,0,0
1,0,1,0,2190.0,1.0,-1.264846,-0.410693,-0.179597,-0.030493,-0.679058,...,0,0,0,1,0,0,0,0,0,0
2,0,1,0,2190.0,1.0,-1.402528,0.495571,-0.179597,-0.030493,0.919715,...,0,0,0,1,0,0,0,1,0,0
3,0,0,0,2555.0,1.0,-1.402528,-1.090391,-0.595687,-0.459889,-0.310110,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,2555.0,1.0,-1.402528,0.042439,0.028448,-0.030493,0.058838,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49963,1,0,0,2555.0,1.0,-1.402528,0.495571,0.236493,0.398904,0.919715,...,0,0,0,1,0,0,0,0,0,0
49964,1,0,0,2491.0,1.0,-1.264846,-0.863825,-1.011777,-0.889285,0.427785,...,0,0,1,1,0,0,0,0,0,0
49965,1,0,0,2451.0,1.0,0.800376,0.269005,0.028448,0.184206,-0.679058,...,0,0,0,0,1,1,0,0,0,0
49966,1,1,0,2555.0,1.0,0.800376,0.269005,0.028448,0.184206,0.181820,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
s, a = 0, 0
ty, sty = Fb_Y[f't_S{s}_A{a}'], Fb_Y[f'St_S{s}_A{a}']
tc, stc = Gb_C[f't_S{s}_A{a}'], Gb_C[f'St_S{s}_A{a}']

t_arr = ty[::10] #
st_arr = sty[::10]

t1 = time()
func = interp1d(t_arr, st_arr, kind='nearest', fill_value='extrapolate')
result, error = quad(func, 0, t_arr.max() + 10, limit=5)

print(f"Time: {time()-t1:.2f} s.")
print(f"Result of integration: {result}, error: {error}")

xnew = np.arange(0, t_arr.max(), 0.1)
ynew = func(xnew)   # use interpolation function returned by `interp1d`
plt.plot(t_arr, st_arr, 'o', xnew, ynew, '--')
plt.show()

In [ ]:
# df_combined = pd.read_csv("/data/whi/data/main_study/processed/whi_combined_nocolinear.csv")  # combined dataset  # combined dataset
# df_corr = df_combined.iloc[:,4:].corr()
# thresh = 0.8
# where_colinear = list()
# corr_arr = np.array(df_corr).copy()
# for i in range(len(df_corr)):
#     dummy_arr = corr_arr[i,:]
#     if np.count_nonzero((dummy_arr > thresh) | (dummy_arr < -thresh)) > 1:
#         where_colinear.append(i)

# co_tuples = list()
# for ind in where_colinear:
#     dummy_arr = corr_arr[ind,:]
#     co_tuples.append(np.where((dummy_arr > thresh) | (dummy_arr < -thresh)))

# dc_list = list()
# for i in range(len(co_tuples)):
#     dc_list.append(co_tuples[i][0][0])

# dc_list = [x for i, x in enumerate(dc_list) if x not in dc_list[:i]]

# disc_cols = list(df_corr.columns[dc_list])

# df_combined = df_combined.drop(columns=disc_cols).reset_index(drop=True)

In [ ]:
# cov_list = []
# with open('data/whi/whi_features_new.txt', 'r') as f:
#     cov_list = f.read().splitlines()

# print(f"Num covs: {len(cov_list)}")

# #X = df_combined[jD["cov_list"]].drop(columns=['DMARM_Intervention', 'DMARM_Not randomized to DM', 'CADARM_Intervention', 'CADARM_Not randomized to DM'])
# X = df_combined[cov_list]
# y = df_combined["S"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# logreg = LogisticRegressionCV(cv=5, random_state=0, penalty='l2', solver='lbfgs', l1_ratios=[0.5])
# logreg.fit(X, y)

# y_train_pred = logreg.predict(X_train)
# y_train_proba = logreg.predict_proba(X_train)

# y_test_pred = logreg.predict(X_test)
# y_test_proba = logreg.predict_proba(X_test)

# train_acc = (y_train_pred == y_train).mean()
# test_acc = (y_test_pred == y_test).mean()

# print(f'Train acc :{train_acc}\nTest acc : {test_acc}')

# plt.hist(logreg.predict_proba(X)[:,1], bins=100)
# plt.show()

# n = 50

# top_coef_indices = np.argsort(np.abs(logreg.coef_[0]))[::-1][:n]
# top_coef_features = X.columns[top_coef_indices]
# top_coef_values = logreg.coef_[0][top_coef_indices]

# # Print the feature names and their coefficients
# for feature, coef in zip(top_coef_features, top_coef_values):
#     print(f'Coeff: {coef:.2f}, Ft: {feature}')